In [8]:
import gurobipy as gp
from gurobipy import GRB
import itertools

# Graph
V = [1, 2, 3, 4, 5, 6]
E = [(1, 2), (2, 3), (1, 5), (2, 4), (2, 6), (3, 5), (3, 6)]

def is_independent_set(S):
    for u, v in itertools.combinations(S, 2):
        if (u, v) in E or (v, u) in E:
            return False
    return True

# Initial stable sets: each vertex alone
stable_sets = [{v} for v in V]

# Initial master problem
master = gp.Model("VertexColoring")
master.setParam("OutputFlag", 0)

lambda_vars = {}
for i, s in enumerate(stable_sets):
    lambda_vars[i] = master.addVar(vtype=GRB.CONTINUOUS, name=f"lambda_{i}")

# Constraint: Each vertex must be covered
constraints = {}
for v in V:
    constraints[v] = master.addConstr(
        gp.quicksum(lambda_vars[i] for i, s in enumerate(stable_sets) if v in s) == 1,
        name=f"cover_{v}"
    )

master.setObjective(gp.quicksum(lambda_vars[i] for i in lambda_vars), GRB.MINIMIZE)

# Column Generation Loop
iteration = 0
while True:
    iteration += 1
    master.optimize()
    
    # Dual values for pricing
    duals = {v: constraints[v].Pi for v in V}

    best_set = None
    best_value = 0

    for r in range(2, len(V) + 1):
        for subset in itertools.combinations(V, r):
            if is_independent_set(subset):
                value = sum(duals[v] for v in subset)
                if value > best_value:
                    best_value = value
                    best_set = set(subset)

    reduced_cost = 1 - best_value

    print(f"Iteration {iteration}: Best reduced cost = {reduced_cost:.4f}")

    if reduced_cost >= -1e-6:
        print("No improving column found. Optimal LP solution reached.")
        break

    stable_sets.append(best_set)
    idx = len(stable_sets) - 1
    new_var = master.addVar(vtype=GRB.CONTINUOUS, name=f"lambda_{idx}")
    lambda_vars[idx] = new_var

    for v in best_set:
        constraints[v].setAttr("rhs", 1)  # Keep RHS at 1
        master.chgCoeff(constraints[v], new_var, 1.0)

    master.update()

# Output 
print("\nFinal LP Solution:")
for i, var in lambda_vars.items():
    if var.X > 1e-6:
        print(f"λ_{i} (set {stable_sets[i]}) = {var.X:.2f}")


Iteration 1: Best reduced cost = -2.0000
Iteration 2: Best reduced cost = -1.0000
Iteration 3: Best reduced cost = -1.0000
Iteration 4: Best reduced cost = -1.0000
Iteration 5: Best reduced cost = 0.0000
No improving column found. Optimal LP solution reached.

Final LP Solution:
λ_5 (set {6}) = 1.00
λ_6 (set {1, 3, 4}) = 1.00
λ_9 (set {2, 5}) = 1.00


In [9]:
import gurobipy as gp
from gurobipy import GRB
import itertools

roll_types = [
    {"length": 15, "cost": 10},
    {"length": 22, "cost": 16}
]

item_lengths = [4, 4, 7]  # lengths of items
demands = [80, 30, 100]   # demand of each item
num_items = len(item_lengths)

# Initial Pattern
patterns = []  # each element: (roll_type_index, item_counts)

for j, roll in enumerate(roll_types):
    for i in range(num_items):
        max_pieces = roll["length"] // item_lengths[i]
        if max_pieces > 0:
            pattern = [0] * num_items
            pattern[i] = max_pieces
            patterns.append((j, pattern))  # roll type index, pattern


# Restricted Master Problem
master = gp.Model("cutting_stock")
master.setParam("OutputFlag", 0)

lambda_vars = []
pattern_info = []  # store tuples (roll_type_index, pattern_list)

for roll_type_idx, pattern in patterns:
    var = master.addVar(obj=roll_types[roll_type_idx]["cost"], vtype=GRB.CONTINUOUS)
    lambda_vars.append(var)
    pattern_info.append((roll_type_idx, pattern))

# Demand constraints
constraints = []
for i in range(num_items):
    constr = master.addConstr(
        gp.quicksum(lambda_vars[p] * pattern_info[p][1][i] for p in range(len(lambda_vars))) >= demands[i]
    )
    constraints.append(constr)

master.update()

def solve_pricing_problem(dual_prices, roll):
    # Maximize sum_i dual[i] * x[i] s.t. sum_i l_i * x[i] <= roll_length
    knapsack = gp.Model("pricing")
    knapsack.setParam("OutputFlag", 0)

    x = [knapsack.addVar(vtype=GRB.INTEGER, lb=0) for _ in range(num_items)]
    knapsack.setObjective(gp.quicksum(dual_prices[i] * x[i] for i in range(num_items)), GRB.MAXIMIZE)
    knapsack.addConstr(gp.quicksum(item_lengths[i] * x[i] for i in range(num_items)) <= roll["length"])
    knapsack.optimize()

    value = knapsack.ObjVal
    pattern = [int(x[i].X) for i in range(num_items)]
    return value, pattern

iteration = 0
while True:
    iteration += 1
    master.optimize()
    duals = [constr.Pi for constr in constraints]

    found_better = False

    for j, roll in enumerate(roll_types):
        value, pattern = solve_pricing_problem(duals, roll)
        reduced_cost = roll["cost"] - value

        print(f"Iter {iteration}, Roll {j+1}: Reduced Cost = {reduced_cost:.4f}")

        if reduced_cost < -1e-6:
            var = master.addVar(obj=roll["cost"], vtype=GRB.CONTINUOUS)
            lambda_vars.append(var)
            pattern_info.append((j, pattern))
            for i in range(num_items):
                constraints[i].setAttr("rhs", demands[i])
                master.chgCoeff(constraints[i], var, pattern[i])
            master.update()
            found_better = True

    if not found_better:
        break

# Output
print("\nFinal LP solution (fractional):")
total_cost = 0
for i, var in enumerate(lambda_vars):
    if var.X > 1e-6:
        roll_idx, pattern = pattern_info[i]
        print(f"Use {var.X:.2f} of Roll {roll_idx+1} with pattern {pattern}")
        total_cost += var.X * roll_types[roll_idx]["cost"]

print(f"Total LP Cost: ${total_cost:.2f}")


Iter 1, Roll 1: Reduced Cost = -1.4000
Iter 1, Roll 2: Reduced Cost = -0.4000
Iter 2, Roll 1: Reduced Cost = -1.4000
Iter 2, Roll 2: Reduced Cost = -0.4000
Iter 3, Roll 1: Reduced Cost = 0.0000
Iter 3, Roll 2: Reduced Cost = 1.0000

Final LP solution (fractional):
Use 22.50 of Roll 1 with pattern [0, 0, 2]
Use 40.00 of Roll 1 with pattern [2, 0, 1]
Use 15.00 of Roll 1 with pattern [0, 2, 1]
Total LP Cost: $775.00


In [10]:
import gurobipy as gp
from gurobipy import GRB
import heapq

arcs = {
    ('s', '1'): {'length': 2},
    ('s', '2'): {'length': 1},
    ('1', 't'): {'length': 3},
    ('2', 't'): {'length': 5},
}
nodes = {'s', '1', '2', 't'}
commodities = [1, 2]
capacity = {a: 2 for a in arcs}

# Initial feasible solution
initial_paths = {
    1: [['s', '1', 't']],
    2: [['s', '2', 't']]
}

def path_cost(path):
    return sum(arcs[(path[i], path[i+1])]['length'] for i in range(len(path)-1))

def arc_usage(path):
    return {a: int(a in zip(path, path[1:])) for a in arcs}

paths = {k: list(p_list) for k, p_list in initial_paths.items()}
model = gp.Model("MultiCommodityFlow_CG")
model.setParam('OutputFlag', 0)

# Lambda variables
lambda_vars = {}
for k in commodities:
    for idx, p in enumerate(paths[k]):
        lambda_vars[(k, idx)] = model.addVar(name=f"lambda_{k}_{idx}")

model.update()

# Objective
model.setObjective(gp.quicksum(path_cost(paths[k][idx]) * lambda_vars[(k, idx)]
                                for k in commodities for idx in range(len(paths[k]))), GRB.MINIMIZE)

# Assignment constraints: each commodity must use exactly one path
assign_constraints = {
    k: model.addConstr(gp.quicksum(lambda_vars[(k, idx)] for idx in range(len(paths[k]))) == 1,
                       name=f"assign_{k}")
    for k in commodities
}

# Arc capacity constraints
cap_constraints = {}
for a in arcs:
    cap_constraints[a] = model.addConstr(
        gp.quicksum(lambda_vars[(k, idx)] * arc_usage(paths[k][idx])[a]
                    for k in commodities for idx in range(len(paths[k]))) <= capacity[a],
        name=f"cap_{a}"
    )

model.optimize()

print("Optimal cost (primal obj) of the restricted master problem:", model.ObjVal)

# Extract dual variables
pi = {k: assign_constraints[k].Pi for k in commodities}   # duals of assignment constraints
sigma = {a: cap_constraints[a].Pi for a in arcs}          # duals of capacity constraints

print("Dual variables for assignment constraints (pi):", pi)
print("Dual variables for capacity constraints (sigma):", sigma)

# Pricing problem: shortest path with reduced costs = arc length - dual capacity prices
def shortest_path_reduced_cost(start, end, arcs, sigma):
    adj = {node: [] for node in nodes}
    for (u, v), attr in arcs.items():
        reduced_cost = attr['length'] - sigma[(u, v)]
        adj[u].append((v, reduced_cost))
    
    # Dijkstra's algorithm
    heap = [(0, start, [start])]  # (cost, node, path)
    visited = set()
    
    while heap:
        cost, node, path = heapq.heappop(heap)
        if node == end:
            return cost, path
        if node in visited:
            continue
        visited.add(node)
        for neighbor, w in adj[node]:
            if neighbor not in visited:
                heapq.heappush(heap, (cost + w, neighbor, path + [neighbor]))
    return float('inf'), []

# Solve pricing problem for each commodity and print reduced cost
for k in commodities:
    cost, path = shortest_path_reduced_cost('s', 't', arcs, sigma)
    reduced_cost = cost - pi[k]
    print(f"Commodity {k}: Shortest path with reduced costs = {path}, reduced cost = {reduced_cost}")


Optimal cost (primal obj) of the restricted master problem: 11.0
Dual variables for assignment constraints (pi): {1: 5.0, 2: 6.0}
Dual variables for capacity constraints (sigma): {('s', '1'): 0.0, ('s', '2'): 0.0, ('1', 't'): 0.0, ('2', 't'): 0.0}
Commodity 1: Shortest path with reduced costs = ['s', '1', 't'], reduced cost = 0.0
Commodity 2: Shortest path with reduced costs = ['s', '1', 't'], reduced cost = -1.0
